### **D2APR: Aprendizado de Máquina e Reconhecimento de Padrões** (IFSP, Campinas) <br/>
**Prof**: Samuel Martins (Samuka) <br/>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>. <br/><br/>

#### Custom CSS style

In [ ]:
%%html
<style>
.dashed-box {
    border: 1px dashed black !important;
#    font-size: var(--jp-content-font-size1) !important;
}

.dashed-box table {

}

.dashed-box tr {
    background-color: white !important;
}
        
.alt-tab {
    background-color: black;
    color: #ffc351;
    padding: 4px;
    font-size: 1em;
    font-weight: bold;
    font-family: monospace;
}
// add your CSS styling here
</style>

<span style='font-size: 2.5em'><b>California Housing 🏡</b></span><br/>
<span style='font-size: 1.5em'>Predict the median housing price in California districts</span>

<span style="background-color: #ffc351; padding: 4px; font-size: 1em;"><b>Sprint #2</b></span>

<img src="./imgs/california-flag.png" width=300/>

---



## Before starting this notebook
This jupyter notebook is designed for **experimental and teaching purposes**. <br/>
Although it is (relatively) well organized, it aims at solving the _target problem_ by evaluating (and documenting) _different solutions_ for somes steps of the **machine learning pipeline** — see the ***Machine Learning Project Checklist by xavecoding***. <br/>
We tried to make this notebook as literally a _notebook_. Thus, it contains notes, drafts, comments, etc.<br/>

For teaching purposes, some parts of the notebook may be _overcommented_. Moreover, to simulate a real development scenario, we will divide our solution and experiments into **"sprints"** in which each sprint has some goals (e.g., perform _feature selection_, train more ML models, ...). <br/>
The **sprint goal** will be stated at the beginning of the notebook.

A ***final notebook*** (or any other kind of presentation) that compiles and summarizes all sprints — the target problem, solutions, and findings — should be created later.

#### Conventions

<ul>
    <li>💡 indicates a tip. </li>
    <li> ⚠️ indicates a warning message. </li>
    <li><span class='alt-tab'>alt tab</span> indicates and an extra content (<i>e.g.</i>, slides) to explain a given concept.</li>
</ul>

---

## 🎯 Sprint Goals
- Remove outliers (those with capped values)
- Run the remaining steps of Sprint #1
---

### 0. Imports and default settings for plotting

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

## 💽 2. Get the Data

### 2.2. Load the Data

In [ ]:
import pandas as pd

housing = pd.read_csv('./datasets/housing.csv')

In [ ]:
housing.head()

### 2.3. Take a quick look at the data structure

In [ ]:
housing.info()

In [ ]:
# plot a histogram for each numeric attibute from the dataframe
housing.hist(bins=50, figsize=(20,15))
display()  # just to avoid texts in the notebook output

### 2.4. Create a **`Test Set`**

### Removing outliers with capped values
By looking at the charts above, we can see that the `housing_median_age` and the `median_house_value` has many **capped values** located at their maximum.

#### **`housing_median_age`**

In [ ]:
housing['housing_median_age'].describe()

In [ ]:
mask = housing['housing_median_age'] > 48
housing.loc[mask, 'housing_median_age'].value_counts()

In [ ]:
sns.histplot(data=housing[mask], x='housing_median_age')

There are many instances with `housing_median_age` equals to **52**.

#### **`median_house_value`**

In [ ]:
housing['median_house_value'].describe()

In [ ]:
mask = housing['median_house_value'] > 480000
housing.loc[mask, 'median_house_value'].value_counts()

In [ ]:
sns.histplot(data=housing[mask], x='median_house_value')

##### **Removing the outliers**

In [ ]:
housing_raw = housing.copy()

In [ ]:
no_outlier_mask = (housing['housing_median_age'] < 52) & (housing['median_house_value'] < 500001)
housing = housing[no_outlier_mask].copy()

In [ ]:
housing.info()

### Segmenting samples by their `median income`

In [ ]:
housing['median_income_group'] = pd.cut(housing['median_income'], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5], include_lowest=True)
housing['median_income_group']

In [ ]:
housing['median_income_group'].value_counts()

In [ ]:
# proportional of the groups in the entire dataset
housing['median_income_group'].value_counts() / len(housing['median_income_group'])

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(housing['median_income_group'])
plt.grid(True)

### Stratified Sampling

In [ ]:
from sklearn.model_selection import train_test_split

housing_train, housing_test = train_test_split(housing, test_size=0.2, stratify=housing['median_income_group'], random_state=42)

In [ ]:
housing_train.head()

In [ ]:
housing_train.shape

In [ ]:
# proportion of the groups in the training set
housing_train['median_income_group'].value_counts() / len(housing_train['median_income_group'])

In [ ]:
housing_test.head()

In [ ]:
housing_test.shape

In [ ]:
# proportion of the groups in the testing set
housing_test['median_income_group'].value_counts() / len(housing_test['median_income_group'])

In [ ]:
# removing the attribute 'median_income_group'
housing_train = housing_train.drop(columns=['median_income_group'])
housing_test = housing_test.drop(columns=['median_income_group'])

#### **Saving datasets**

In [ ]:
# if you want to keep the reference for the row indices from the original dataset, remove the index=False
housing_train.to_csv('./datasets/housing_train_sprint-2.csv', index=False)
housing_test.to_csv('./datasets/housing_test_sprint-2.csv', index=False)

## 🧹 3. Data Cleaning

### 3.1. Missing values

In [ ]:
housing_train.info()

In [ ]:
# missing the `total_bedrooms` values with its median

housing_train_clean = housing_train.copy()

median_total_bedrooms = housing_train_clean['total_bedrooms'].median()
housing_train_clean['total_bedrooms'].fillna(median_total_bedrooms, inplace=True)

In [ ]:
housing_train_clean.info()

## 📊 4. Explore the Data
We next perform a _simple exploratory data analysis (EDA)_ to gain insights about the data. <br/>
A more complete EDA with hypotheses about the problem should be further elaborated. 

### 4.1. Visualizing Geographical Data

In [ ]:
### To install plotly
# pip install plotly
# jupyter labextension install jupyterlab-plotly

### Plotly Maps
# https://plotly.com/python/scattermapbox/
# To plot on Mapbox maps with Plotly you may need a Mapbox account and a public Mapbox Access Token - https://www.mapbox.com/studio

import plotly.express as px

px.set_mapbox_access_token('pk.eyJ1IjoiY2llbmNpYWRlZGFkb3NpZnNwY2FtcGluYXMiLCJhIjoiY2tzcW9sNTRhMGR2bzJ1cGcxNTI1bWppdiJ9.4TJwkUhuLIt-2nH0YudsMg')
fig = px.scatter_mapbox(housing_train_clean, lat="latitude", lon="longitude", color="median_house_value", size="population",
                        color_continuous_scale=px.colors.sequential.Viridis, size_max=15, zoom=5, width=1000, height=800)
fig.show()

When removing the most expensive houses in the dataset -- those with *capped values* at \\$ 500,001.00 -- our map has changed just a little bit, nothing to worry.

The (obvious) findings keep the same:
- There small and big districts (in term of population) close and far from the coast
- The most expensive houses are very close to the coast

### 4.2. Looking for Correlations

In [ ]:
# plotting the correlation coeficients as a heatmap
plt.figure(figsize=(16, 6))
mask = np.triu(np.ones_like(housing_train_clean.corr(), dtype=np.bool))  # creates a triangular matrix based on the pandas correlation matrix

heatmap = sns.heatmap(housing_train_clean.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16);

In [ ]:
housing_train_clean.corr()["median_house_value"].sort_values(ascending=False)

The correlation has slightly changed. The _most promising attribute_ to predict the `median house value` keeps being the **`median income`**.

In [ ]:
plt.figure(figsize=(15, 8))
sns.lmplot(data=housing_train_clean, x="median_income", y="median_house_value", aspect=2, height=8, scatter_kws={'alpha': 0.3}, line_kws={'color': 'r'})
plt.yticks(range(0, 700001, 25000))
plt.grid(True)
plt.title('Median income vs Median house value')
display()

This plot is better than that from Sprint #1.

Some findings:
- The _correlation_ is **strong**: see the regression line and the _upward trend_
- There is a horizontal line around \\$350,000, another around \\$450,000, perhaps a few more.
  - You may want to try removing the corresponding districts to prevent your algorithms from learning to reproduce these data quirks.
  - For now, we will not remove them.

In [ ]:
# Zoom in the stats for the target outcome
housing_train_clean['median_house_value'].describe()

## 🛠️ 5. Prepare the Data

In [ ]:
housing_train_clean.head()

#### **Separating the independent variables (features) and the _dependent variable_ (target outcome)**

In [ ]:
housing_train_pre = housing_train_clean.drop(columns=['median_house_value'])
housing_train_target = housing_train_clean['median_house_value'].copy()

<table align="left" class="dashed-box">
<tr>
    <td>⚠️</td>
    <td>Remember to merge the <i>features</i> and the <i>target outcome</i> into a single dataframe before saving it to disk.</td>
</tr>
</table><br/><br/>

### 5.1. Categorical Variabel Encoding

In [ ]:
# one hot encoding by pandas
housing_train_pre = pd.get_dummies(data=housing_train_pre, columns=['ocean_proximity'])
housing_train_pre.head()

#### **Saving the pre-processed training set**

In [ ]:
housing_train_pre_saving = housing_train_pre.copy()
housing_train_pre_saving['median_house_value'] = housing_train_target
housing_train_pre_saving.to_csv('./datasets/housing_train_pre_sprint-2.csv', index=False)

## 🏋️‍♀️ 6. Train ML Algorithms

### 6.1. Getting the independent (features) and dependent variables (outcome)

In [ ]:
X_train = housing_train_pre.values
y_train = housing_train_target.values

### 6.1. Training the Models

In [ ]:
from sklearn.linear_model import LinearRegression

linear_regressor = LinearRegression()  # default parameters
linear_regressor.fit(X_train, y_train)

### 6.3. Evaluating on the Training Set

### **Prediction**

In [ ]:
y_train_pred = linear_regressor.predict(X_train)

#### **Evaluation**

In [ ]:
r2_score = linear_regressor.score(X_train, y_train)
print(f'R² = {r2_score}')

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_train, y_train_pred, squared=False)
print(f'RMSE = {rmse}')

Although the RMSE has decreased (\\$58,689) compared to the Sprint #1 (\\$69,050), the current model *is not necessarily better*. <br/>
The reason is that we now have a _smaller dataset_ that has generated _different_ training and testing sets from Sprint #1.

However, we still have a high error on the training set, which indicates **underfitting**.

### **Visual Analysis**

##### **Prediction vs Real**

In [ ]:
sns.scatterplot(x=y_train_pred, y=y_train)
plt.xlabel('Prediction')
plt.ylabel('Real')
plt.title('Median housing value - Prediction vs Real')

##### **Residual Analysis**
Plot of Prediction vs Residual. This analysis is interesting because we can detect if we meet the assumption of **homoscedasticity**.

<img src='./imgs/residual-analysis.png' width=600/>

In [ ]:
residual = y_train - y_train_pred

In [ ]:
sns.scatterplot(x=y_train_pred, y=residual)
plt.xlabel('Prediction')
plt.ylabel('Residual')
plt.title('Median housing value - Prediction vs Residual')

we have removed that diagonal top line in this plot. However, the model still **does not** meet the **homoscedasticity**.

In [ ]:
sns.histplot(residual)

The residual keeps roghly following a _normal distribution_.